<center><img src="../Picture Data/logo.png" alt="Header" style="width: 800px;"/></center>

In [ ]:
import ipywidgets.widgets as widgets
controller = widgets.Controller(index=0)  #Replace with the index number of the controller you just tested
display(controller)

Import some module

In [ ]:
#Import function library path
from servoserial import ServoSerial
from jetbot import Robot
import traitlets
from jetbot import Camera
from jetbot import bgr8_to_jpeg
from jetbot import Heartbeat
import threading
import time
# Thread function operation library
import inspect
import ctypes

import traitlets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg
import os
from uuid import uuid1
import cv2
import numpy as np
from RGB_Lib import Programing_RGB
RGB = Programing_RGB()

Create and open a display camera initialization instance

In [ ]:
camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)  # this width and height doesn't necessarily have to match the camera
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)
display(image)

Add heartbeat connection

In [ ]:
def handle_heartbeat_status(change):
    if change['new'] == Heartbeat.Status.dead:
        camera_link.unlink()
        robot.stop()

heartbeat = Heartbeat(period=0.5)
# Append the callback function to the heartbeat state
heartbeat.observe(handle_heartbeat_status, names='status')

Create a method to actively stop the process

In [ ]:
def _async_raise(tid, exctype):
    """raises the exception, performs cleanup if needed"""
    tid = ctypes.c_long(tid)
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
    if res == 0:
        raise ValueError("invalid thread id")
    elif res != 1:
        # """if it returns a number greater than one, you're in trouble,
        # and you should call it again with exc=NULL to revert the effect"""
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
        
def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)

Method for creating up, down, left, and right movements of a PTZ camera separately

In [ ]:
'''Servo Part'''
servo_device = ServoSerial() 
def camUpFunction():
    global updownpulse
    updownpulse+=15
    if updownpulse>4095:
        updownpulse=4095
    servo_device.Servo_serial_control(2, updownpulse)

def camDownFunction():
    global updownpulse
    updownpulse-=15
    if updownpulse<1300:
        updownpulse=1300
    servo_device.Servo_serial_control(2, updownpulse)

def camLeftFunction():
    global leftrightpulse
    leftrightpulse+=15
    if leftrightpulse>3600:
        leftrightpulse=3600
    servo_device.Servo_serial_control(1, leftrightpulse)

def camRightFunction():
    global leftrightpulse
    leftrightpulse-=15
    if leftrightpulse<600:
        leftrightpulse=600
    servo_device.Servo_serial_control(1, leftrightpulse)

def camservoInitFunction():
    global leftrightpulse, updownpulse
    leftrightpulse = 2048
    updownpulse = 2048
    servo_device.Servo_serial_control(1, 2048)
    time.sleep(0.1)
    servo_device.Servo_serial_control(2, 2048)

### Initialize the position of the PTZ camera
Run the following cell code to initialize the PTZ project to the initial location

In [ ]:
camservoInitFunction()

### Load the Robot class
This class allows us to easily control the JetBot motor

In [ ]:
robot = Robot()

### Create onboard breathing light method

In [ ]:
def BLN_Onboard():
    global i , k
    i = k = 0
    while True:
        if k == 0:
            robot.set_bln(i)
            i += 0.01
            if( i >= 1 ):
                k = 1
            time.sleep(0.005)
        elif k == 1:
            robot.set_bln(i)
            i -=0.01
            if i <= 0 :
                k = 0
            time.sleep(0.005)

Turn on the onboard breathing light independent process by running the cell code below

In [ ]:
thread1 = threading.Thread(target=BLN_Onboard)
# thread1.setDaemon(True)
thread1.start()

<center><img src="../Picture Data/Handle.png" alt="Header" style="width: 400px;"/></center>

### Create a handle to control the movement of Jetbot in real time
### Please use the ANOLOG button in the middle of the handle to switch to the simulation mode before using.

Program features:
          1. Left rocker control Jetbot movement, right rocker control the servo movement
          2. Press the SELECT button to reset the PTZ angle
          3. Press the L side button No.1 to control the PTZ rise, press the L side button No.2 to control the gimbal decline.

In the default code, the Yahboom handle is used by default. For other handles, please refer to the key table to change the value.
     
     If you are using Yahboom accessory, please use the code section ---1
     If you are using the Xbox360's handle, please use the code section ---2

In [ ]:
controller.axes

In [ ]:

lower_red = np.array([150,50,50])
upper_red = np.array([180,255,255])

lower_green = np.array([50,150,50])
upper_green = np.array([80,255,255])

lower_blue = np.array([100,100,120])
upper_blue = np.array([150,255,255])


In [ ]:
def jetbot_motion():
    count1 = count2 = count3 = count4 =  count5 = 0
    while 1:
        
        if controller.axes[1].value <= 0.1:
            if (controller.axes[0].value <= 0.1 and controller.axes[0].value >= -0.1 
                and controller.axes[1].value <= 0.1 and controller.axes[1].value >= -0.1):
                robot.stop()
            else:
                robot.set_motors(-controller.axes[1].value + controller.axes[0].value, -controller.axes[1].value - controller.axes[0].value)
            
            time.sleep(0.01)
        else:
            robot.set_motors(-controller.axes[1].value - controller.axes[0].value, -controller.axes[1].value + controller.axes[0].value)
            time.sleep(0.01)
        if controller.axes[3].value == 1:
            count1  += 1
            if count1 >= 3:
                camDownFunction()
                count1 = 0
        elif controller.axes[3].value == -1:
            count1  += 1
            if count1 >= 3:
                camUpFunction()
                count1 = 0
        else:
            count1 = 0
        if controller.axes[2].value == 1:
            count2  += 1
            if count2 >= 3:
                camRightFunction()
                count2 = 0
        elif controller.axes[2].value == -1:
            count2  += 1
            if count2 >= 3:
                camLeftFunction()
                count2 = 0
        else:
            count2 = 0
        #Servo control camera up down, left and right is reset
        if controller.buttons[8].value == True:
            count3 += 1
            if count3 >= 3:
                camservoInitFunction()
                count3 = 0
        else:
            count3 = 0
        
        #Servo control servo rise and decline
        if controller.buttons[6].value == True:
            count4 += 1
            if count4 >= 3:
                robot.set_vertical_motors(1)
                count4 = 0
        elif controller.buttons[4].value == True:
            count4 += 1
            if count4 >= 3:
                robot.set_vertical_motors(-1)
                count4 = 0
        else:
            robot.set_vertical_motors(0)
            count4 = 0
            
            
            
       

In [ ]:
def jetbot_color():
     
    while 1:
        frame = camera.value
        hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)

        mask = cv2.inRange(hsv,lower_red,upper_red)
        mask1 = cv2.inRange(hsv,lower_green,upper_green)
        mask2 = cv2.inRange(hsv,lower_blue,upper_blue)

        b = np.array([mask.sum(),mask1.sum(),mask2.sum()])

        color_list = np.array([[0xFF, 0x00, 0x00],[0x00, 0xFF, 0x00],[0x00, 0x00, 0xFF]])

        bg = b.argmax()

        RGB.Set_All_RGB(color_list[bg][0],color_list[bg][1],color_list[bg][2])

In [ ]:
while 1:
        frame = camera.value
        hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)

        mask = cv2.inRange(hsv,lower_red,upper_red)
        mask1 = cv2.inRange(hsv,lower_green,upper_green)
        mask2 = cv2.inRange(hsv,lower_blue,upper_blue)

        b = np.array([mask.sum(),mask1.sum(),mask2.sum()])

        color_list = np.array([[0xFF, 0x00, 0x00],[0x00, 0xFF, 0x00],[0x00, 0x00, 0xFF]])

        bg = b.argmax()

        RGB.Set_All_RGB(color_list[bg][0],color_list[bg][1],color_list[bg][2])

Control the independent process of Jetbot motion in real time by running the cell code below to open the handle

In [ ]:
thread2 = threading.Thread(target=jetbot_motion)
thread2.setDaemon(True)
thread2.start()

In [ ]:
thread3 = threading.Thread(target=jetbot_color)
thread3.setDaemon(True)
thread3.start()

Ending onboard breathing RGB process

In [ ]:
stop_thread(thread1)

Ending Jetbot movement process

In [ ]:
stop_thread(thread2)
robot.stop()

In [ ]:
camera.stop()